Installed packages in virtual environment

In [ ]:
# %pip install pymongo
# %pip install pandas (will install numpy)
# %pip install google-cloud-bigquery


In [1]:
import pandas as pd
from pymongo import MongoClient
import numpy as np
import warnings

warnings.filterwarnings("ignore")

## 1. init connection to database
- Create 2 collections: business, reviews
- upload json files into their respective collections

In [2]:
client = MongoClient()
client = MongoClient('localhost', 27017)

In [3]:
yelp_db = client["Yelp_dataset"] 
business_collection = yelp_db['business'] 
review_collection = yelp_db['reviews']

## 2. Initialise tables

Currently, we need 4 tables
- Business table: { Business_id (PK) , Name , State, City , Stars, Review_count, weighted_review, Categories }
- Names table : {Name (PK) , Number of outlets, Average reviews , Total reviews} 
- Reviews table {Review_id (PK), Business_id (FK), Stars, Text}
- Categories table : {Category, Count (number of appearance), score (avg)}




TODO: calculate weighted reviews: https://medium.com/district-data-labs/computing-a-bayesian-estimate-of-star-rating-means-651496a890ab
https://www.codementor.io/@arpitbhayani/solving-an-age-old-problem-using-bayesian-average-15fy4ww08p#cumulative-rating 
https://www.algolia.com/doc/guides/managing-results/must-do/custom-ranking/how-to/bayesian-average/ 

#### a. Create a business dataframe

In [4]:
business_records = []
i = 0
#mongoQuery = {'$or' : [{'categories' : {'$regex' : '.*Restaurants.*'}},{'categories' : {'$regex' : '.*Food.*'}}]}
for row in business_collection.find():
    cleaned_business = {}
    cleaned_business['business_id'] = row['business_id']
    cleaned_business['name'] = row['name']
    if row['city'] == 'San Francisco':
        i += 1
    cleaned_business['city'] = row['city']
    cleaned_business['state'] = row['state']
    cleaned_business['review'] = row['stars']
    cleaned_business['total_reviews'] = row['review_count']
    cleaned_business['categories'] = row['categories']
    business_records.append(cleaned_business)

In [5]:
business_df = pd.DataFrame(
    business_records,
    columns=[
        "business_id",
        "name",
        "city",
        "state",
        "review",
        "total_reviews",
        "categories"
    ])

business_df

,business_id,name,city,state,review,total_reviews,categories
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ",Santa Barbara,CA,5.0,7,"Doctors, Traditional Chinese Medicine, Naturop..."
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,Affton,MO,3.0,15,"Shipping Centers, Local Services, Notaries, Ma..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,Tucson,AZ,3.5,22,"Department Stores, Shopping, Fashion, Home & G..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,Philadelphia,PA,4.0,80,"Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,Green Lane,PA,4.5,13,"Brewpubs, Breweries, Food"
...,...,...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,Edmonton,AB,3.0,13,"Nail Salons, Beauty & Spas"
150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,Nashville,TN,4.0,5,"Pets, Nurseries & Gardening, Pet Stores, Hobby..."
150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,Indianapolis,IN,3.5,8,"Shopping, Jewelry, Piercing, Toy Stores, Beaut..."
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,Edwardsville,IL,4.0,24,"Fitness/Exercise Equipment, Eyewear & Optician..."


In [15]:
business_df['name'].isnull().values.any()

False

In [19]:
business_df['state'].value_counts()

PA     34039
FL     26330
TN     12056
IN     11247
MO     10913
LA      9924
AZ      9912
NJ      8536
NV      7715
AB      5573
CA      5203
ID      4467
DE      2265
IL      2145
TX         4
CO         3
WA         2
HI         2
MA         2
NC         1
UT         1
MT         1
MI         1
SD         1
XMS        1
VI         1
VT         1
Name: state, dtype: int64

#### b. Filter out all food establishments to create a food dataframe

In [6]:
food_df  = business_df[(business_df['categories'].str.contains(pat = 'Food', regex = True)) 
                             | (business_df['categories'].str.contains(pat = 'Restaurants', regex = True))]
food_df

,business_id,name,city,state,review,total_reviews,categories
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,Philadelphia,PA,4.0,80,"Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,Green Lane,PA,4.5,13,"Brewpubs, Breweries, Food"
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,Ashland City,TN,2.0,6,"Burgers, Fast Food, Sandwiches, Food, Ice Crea..."
8,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,Affton,MO,3.0,19,"Pubs, Restaurants, Italian, Bars, American (Tr..."
9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,Nashville,TN,1.5,10,"Ice Cream & Frozen Yogurt, Fast Food, Burgers,..."
...,...,...,...,...,...,...,...
150327,cM6V90ExQD6KMSU3rRB5ZA,Dutch Bros Coffee,Boise,ID,4.0,33,"Cafes, Juice Bars & Smoothies, Coffee & Tea, R..."
150328,1jx1sfgjgVg0nM6n3p0xWA,Savaya Coffee Market,Oro Valley,AZ,4.5,41,"Specialty Food, Food, Coffee & Tea, Coffee Roa..."
150336,WnT9NIzQgLlILjPT0kEcsQ,Adelita Taqueria & Restaurant,Philadelphia,PA,4.5,35,"Restaurants, Mexican"
150339,2O2K6SXPWv56amqxCECd4w,The Plum Pit,Aston,DE,4.5,14,"Restaurants, Comfort Food, Food, Food Trucks, ..."


#### c. group the food establishments by name

In [13]:
name_df = food_df.groupby(['name']).agg({'review' : 'mean', 'total_reviews' : 'sum', 'name' : 'count'}
                                            ).rename(columns = {'review' : 'avg_review','name' : 'total_outlets'}
                                                     ).sort_values(by=['total_outlets'], ascending=False).reset_index()
name_df.head()

,name,avg_review,total_reviews,total_outlets
0,Starbucks,3.126381,20692,724
1,McDonald's,1.863442,17359,703
2,Dunkin',2.302941,9864,510
3,Subway,2.586057,4123,459
4,Taco Bell,2.154795,8325,365


## 3. Load data into warehouse
https://cloud.google.com/bigquery/docs/samples/bigquery-load-table-dataframe 

In [ ]:
%pip install git+https://github.com/googleapis/python-bigquery-pandas.git

In [1]:
from google.cloud import bigquery
from pandas.io import gbq
import pandas_gbq

In [2]:
project_id = 'yelp_data_warehouse'
data_set_id = 'food_dataset'
table = 'test1'
table_id = '{}.{}.{}'.format(project_id, data_set_id, table)

In [ ]:
# Construct a BigQuery client object.
client = bigquery.Client()

job_config = bigquery.LoadJobConfig(
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    write_disposition="WRITE_TRUNCATE",
)

job = client.load_table_from_dataframe(
    food_df, table_id, job_config=job_config
)  # Make an API request.
job.result()  # Wait for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

In [14]:
food_df.to_gbq(destination_table='food_dataset.test1',
               project_id='yelp_data_warehouse',
               if_exists='replace')

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=725825577420-unm2gnkiprugilg743tkbig250f4sfsj.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fpydata-google-auth.readthedocs.io%2Fen%2Flatest%2Foauth.html&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=W35tYPbTBRWt2D4F2NlD8AWAtMEU5i&access_type=offline
